In [2]:

from contextlib import ExitStack

import numpy as np

from dolfinx import la
from dolfinx.mesh import CellType, create_box, locate_entities_boundary

from dolfinx.fem import (DirichletBC, Function, VectorFunctionSpace,
                         apply_lifting, assemble_matrix, assemble_vector,
                         locate_dofs_geometrical, set_bc)
from dolfinx.io import XDMFFile
from dolfinx.mesh import CellType, GhostMode, create_box
from ufl import (Identity, SpatialCoordinate, TestFunction, TrialFunction,
                 as_vector, dx, grad, inner, sym, tr,CellVolume)
from dolfinx.fem import (Constant, DirichletBC, Function, LinearProblem, FunctionSpace, VectorFunctionSpace, 
                         locate_dofs_topological)
from petsc4py.PETSc import ScalarType
import dolfinx
import ufl
from mpi4py import MPI
from petsc4py import PETSc

In [3]:
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
print(f"size is {size}")
print(f"rank is {rank}")

size is 1
rank is 0


# Create mesh model
Steps are below
1. Create Mesh
2. Defince elastic coordinates
3. add gravitational force to each cell
4. define stress/strain for the equations
5. Follow elasticity demo and build the curves

In [4]:
mesh = create_box(
    comm, [np.array([0.0, 0.0, 0.0]),
                     np.array([2.0, 1.0, 0.1])], [100, 100, 15],
    CellType.tetrahedron, GhostMode.shared_facet)


# Rotation rate and mass density
# omega = 300.0
# rho = 10.0
rho = 2700 # density

# Loading due to centripetal acceleration (rho*omega^2*x_i)
x = SpatialCoordinate(mesh)
vol = CellVolume(mesh)
# f = as_vector((0.0, 0.0, -rho*vol))
f = Constant(mesh, ScalarType((0, 0, -rho*1.1*12)))

# Elasticity parameters
E = 1.0e9
nu = 0.0
mu = E / (2.0 * (1.0 + nu))
lmbda = E * nu / ((1.0 + nu) * (1.0 - 2.0 * nu))


def epsilon(u):
    return ufl.sym(ufl.grad(u)) # Equivalent to 0.5*(ufl.nabla_grad(u) + ufl.nabla_grad(u).T)
def sigma(u):
    return lmbda * ufl.nabla_div(u) * ufl.Identity(u.geometric_dimension()) + 2*mu*epsilon(u)

from petsc4py.PETSc import ScalarType
T = Constant(mesh, ScalarType((0, 0, 0)))
ds = ufl.Measure("ds", domain=mesh) # displacement
# Create function space
V = VectorFunctionSpace(mesh, ("Lagrange", 1))

# Define variational problem
u = TrialFunction(V)
v = TestFunction(V)
a = inner(sigma(u), grad(v)) * dx
# L = inner(f, v) * dx
L = inner(f, v) * ufl.dx + inner(T, v) * ds

# Apply the boundary condition
Apply a Dirichlet(fixed) boundary condition to the x and y dimensions of the facets using the np.isclose command to filter through the domain

In [5]:
def clamped_boundary(x):
    return (np.isclose(x[0], 0)) | (np.isclose(x[0],2.0)) | (np.isclose(x[1],1.0)) | (np.isclose(x[1],0.0))

fdim = mesh.topology.dim - 1
boundary_facets = locate_entities_boundary(mesh, fdim, clamped_boundary)

u_D = Function(V)
u_D.x.array[:] = 0
bc = DirichletBC(u_D, locate_dofs_topological(V, fdim, boundary_facets))

In [14]:
problem = LinearProblem(a, L, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu",'pc_factor_mat_solver_type': 'mumps'})
uh = problem.solve()

In [15]:
import pyvista
from dolfinx.plot import create_vtk_topology
# Start xvfb for rendering plots
pyvista.start_xvfb(wait=0.05)

# Create plotter and pyvista grid
p = pyvista.Plotter(title="Deflection", window_size=[800, 800])
topology, cell_types = create_vtk_topology(mesh, mesh.topology.dim)
grid = pyvista.UnstructuredGrid(topology, cell_types, mesh.geometry.x)

# Attach vector values to grid and warp grid by vector
grid["u"] = uh.compute_point_values().real 
actor_0 = p.add_mesh(grid, style="wireframe", color="k")
warped = grid.warp_by_vector("u", factor=1.5)
actor_1 = p.add_mesh(warped)
p.show_axes()
if not pyvista.OFF_SCREEN:
   p.show()
else:
    figure_as_array = p.screenshot("deflection.png")

ViewInteractiveWidget(height=800, layout=Layout(height='auto', width='100%'), width=800)